In [ ]:
# dataset (Husary)
!cp /kaggle/input/last-three-juz/Husary_64kbps_28-30.ayt Husary_64kbps_28-30.zip
!mkdir Husary
!unzip Husary_64kbps_28-30.zip -d Husary

In [ ]:
# dataset (Hudhaify)
!cp /kaggle/input/last-three-juz/Hudhaify_64kbps_28-30.ayt Hudhaify_64kbps_28-30.zip
!mkdir Hudhaify
!unzip Hudhaify_64kbps_28-30.zip -d Hudhaify

In [ ]:
# dataset (Minshawy)
!cp /kaggle/input/last-three-juz/Minshawy_Murattal_128kbps_28-30.ayt Minshawy_128kbps_28-30.zip
!mkdir Minshawy
!unzip Minshawy_128kbps_28-30.zip -d Minshawy

In [ ]:
# requirements
!pip install jiwer
!pip install evaluate
!sudo apt update -y && sudo apt install ffmpeg -y
!pip install -U openai-whisper
!pip install pyarabic

In [11]:
!echo sorah,ayah,wer_percent > /kaggle/working/output/Husary_28_per_ayah.csv
!echo sorah,wer_percent > /kaggle/working/output/Husary_28_per_sorah.csv
!echo wer_percent > /kaggle/working/output/Husary_28_total.csv

!echo sorah,ayah,wer_percent > /kaggle/working/output/Hudhaify_28_per_ayah.csv
!echo sorah,wer_percent > /kaggle/working/output/Hudhaify_28_per_sorah.csv
!echo wer_percent > /kaggle/working/output/Hudhaify_28_total.csv

!echo sorah,ayah,wer_percent > /kaggle/working/output/Minshawy_28_per_ayah.csv
!echo sorah,wer_percent > /kaggle/working/output/Minshawy_28_per_sorah.csv
!echo wer_percent > /kaggle/working/output/Minshawy_28_total.csv

!echo sorah,ayah,wer_percent > /kaggle/working/output/28_per_ayah.csv
!echo sorah,wer_percent > /kaggle/working/output/28_per_sorah.csv
!echo wer_percent > /kaggle/working/output/28_total.csv

In [19]:
!ls /kaggle/input/last-three-juz-text

ayat_28-30.csv


In [12]:
import warnings
import whisper
import pyarabic.araby as araby
import csv
import evaluate
import itertools

shekhs = {
    "Husary": {"name": "Husary_64kbps"},
    "Hudhaify": {"name": "Hudhaify_64kbps"},
    "Minshawy": {"name": "Minshawy_Murattal_128kbps"}
}

for shekh in shekhs:
    data = shekhs[shekh]
    data["per_sorah_file"] = open(f"/kaggle/working/output/{shekh}_28_per_sorah.csv", "a+")
    data["per_ayah_file"] = open(f"/kaggle/working/output/{shekh}_28_per_ayah.csv", "a+")
    data["total_file"] = open(f"/kaggle/working/output/{shekh}_28_total.csv", "a+")

per_ayah = open("/kaggle/working/output/28_per_ayah.csv", "a+")
per_sorah = open("/kaggle/working/output/28_per_sorah.csv", "a+")

warnings.filterwarnings("ignore")
model = whisper.load_model("medium")
wer = evaluate.load("wer")
num_of_sorahs = 9
references_per_sorah = {shekh: list() for shekh in shekhs}
predictions_per_sorah = {shekh: list() for shekh in shekhs}
total_references = {shekh: list() for shekh in shekhs}
total_predictions = {shekh: list() for shekh in shekhs}
sorahs = set()
first_time = True

with open("/kaggle/input/last-three-juz-text/ayat_28-30.csv") as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)
    for line in csv_reader:
        sorah = int(line[0])
        ayah = int(line[1])
        text = line[2]
        if not sorah in sorahs:
            num_of_sorahs -= 1
            sorahs.add(sorah)
            if first_time:
                first_time = False
            else:
                # write sorah
                for shekh in shekhs:
                    wer_sorah = wer.compute(references=references_per_sorah[shekh], predictions=predictions_per_sorah[shekh])
                    shekhs[shekh]["per_sorah_file"].write(f"{sorah - 1},{wer_sorah * 100:.2f}\n")
    
                sorah_ref = list(itertools.chain(*references_per_sorah.values()))
                sorah_pred = list(itertools.chain(*predictions_per_sorah.values()))
                wer_sorah = wer.compute(references=sorah_ref, predictions=sorah_pred)
                per_sorah.write(f"{sorah - 1},{wer_sorah * 100:.2f}\n")
                
                for shekh in shekhs:
                    references_per_sorah[shekh].clear()
                    predictions_per_sorah[shekh].clear()
                
            if num_of_sorahs == -1:
                break
        
        for shekh in shekhs:
            result = model.transcribe(f"/kaggle/working/{shekh}/audio/{shekhs[shekh]['name']}/{sorah:03}{ayah:03}.mp3")
            output_text = araby.strip_diacritics(result["text"])
            wer_ayah = wer.compute(references=[text], predictions=[output_text])
            shekhs[shekh]["per_ayah_file"].write(f"{sorah},{ayah},{wer_ayah * 100:.2f}\n")
            print(output_text)
            predictions_per_sorah[shekh].append(output_text)
            references_per_sorah[shekh].append(text)
            total_predictions[shekh].append(output_text)
            total_references[shekh].append(text)

        wer_ayah = wer.compute(references=[ayat[-1] for ayat in references_per_sorah.values()], predictions=[ayat[-1] for ayat in predictions_per_sorah.values()])
        per_ayah.write(f"{sorah},{ayah},{wer_ayah * 100:.2f}\n")


for shekh in shekhs:
    data = shekhs[shekh]
    wer_total = wer.compute(references=total_references[shekh], predictions=total_predictions[shekh])
    data["total_file"].write(f"{wer_total * 100:.2f}\n")


with open("/kaggle/working/output/28_total.csv", "a+") as total_file:
    total_ref = list(itertools.chain(*total_references.values()))
    total_pred = list(itertools.chain(*total_predictions.values()))
    wer_total = wer.compute(references=total_ref, predictions=total_pred)
    total_file.write(f"{wer_total * 100:.2f}\n")

for shekh in shekhs:
    data = shekhs[shekh]
    data["per_sorah_file"].close()
    data["per_ayah_file"].close()
    data["total_file"].close()

per_ayah.close()
per_sorah.close()

 قد سمع الله قولا التي تجادلك في زوجها وتشتكي إلى الله والله يسمع تحاوركما إن الله سميع بصير
 168. قد سمع الله قولا التي تجابلك في زوجها وتشتكي إلى الله والله يسمع تحاوركم انا الله سميع بصير
 168- قد سمع الله قولا التي تجادلك في زوجها وتشتكي إلى الله والله يسمع تحاوركم إن الله سميع بصير
 168. الذين يظاهرون منكم من نسائهم ما هن أمهاتهم 169. إن أمهاتهم إلا الله ولدنهم وإنهم ليقولونهم كرا من القول وزورا 169. الله العفو وفور
 168. الذين يظاهرون منكم من نسائهم ما هن أمهاتهم إن أمهاتهم إلا لا إي ولدنهم 169. وإنهم ليقولون منكرا من القول وزورا وإن الله لعفو غفور
 168. الذين يظاهرون منكم من نسائهم ما هن أمهاتهم إن أمهاتهم إلا الله ولدنهم وإنهم ليقولون منكرا من القول وزورا 169. وإن الله العفو الغفور
 168. والذين يظاهرون من نسائهم ثم يعودون لما قالوا فتحرير رقبة من قبل أن يتمسق ذلكم توعظون به والله بما تعملون خبير
 148. والذين يظاهرون من نسائهم ثم يعودون لما قالوا فتحرير رقبة من قبل أن يتماسا 149. ذلكم توعظون به والله بما تعملون خبير
 148. والذين يظاهرون من نسائهم ثم يعودون لما قالوا فتحرير رقبة 

In [13]:
!zip -r output.zip output

updating: output/ (stored 0%)
updating: output/Husary_28_per_ayah.csv (deflated 63%)
updating: output/Minshawy_28_per_ayah.csv (deflated 62%)
updating: output/28_total.csv (stored 0%)
updating: output/Hudhaify_28_total.csv (stored 0%)
updating: output/28_per_ayah.csv (deflated 59%)
updating: output/Hudhaify_28_per_ayah.csv (deflated 61%)
updating: output/Hudhaify_28_per_sorah.csv (deflated 16%)
updating: output/Husary_28_per_sorah.csv (deflated 19%)
updating: output/Minshawy_28_total.csv (stored 0%)
updating: output/Minshawy_28_per_sorah.csv (deflated 14%)
updating: output/Husary_28_total.csv (stored 0%)
updating: output/28_per_sorah.csv (deflated 17%)


In [10]:
for shekh in shekhs:
    data = shekhs[shekh]
    data["per_sorah_file"].close()
    data["per_ayah_file"].close()
    data["total_file"].close()

per_ayah.close()
per_sorah.close()